In [18]:
import pandas as pd
import glob

# Get a list of all xlsx files in the "Resources" folder
file_paths = glob.glob('Resources/2023*.xlsx')

# Initialize an empty list to store individual DataFrames
dfs = []

# Iterate over each file path and read the Excel file into a DataFrame
for file_path in file_paths:
    df = pd.read_excel(file_path, engine='openpyxl')
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Rename the 'ShortSqueeze.com Short Interest Data' column
combined_df.rename(columns={'ShortSqueeze.com Short Interest Data': 'Company Name'}, inplace=True)

# Drop unnecessary columns
columns_to_drop = ['Total Short Interest', 'Days to Cover', 'Performance (52-wk)', 'Short: Prior Mo', '% Change Mo/Mo', 'Shares: Float',
                   'Avg. Daily Vol.', 'Shares: Outstanding', 'Short Squeeze Ranking™', '% from 52-wk High', '(abs)',
                   '% from 200 day MA', '(abs).1', '% from 50 day MA', '(abs).2', '% Insider Ownership', '% Institutional Ownership']

# Check if the columns exist in the dataframe before dropping them
columns_to_drop = [col for col in columns_to_drop if col in combined_df.columns]

combined_df.drop(columns_to_drop, axis=1, inplace=True)

# Convert 'Short % of Float' column to numeric
combined_df['Short % of Float'] = pd.to_numeric(combined_df['Short % of Float'], errors='coerce')

# Filter by Short % of Float >= 20
combined_df = combined_df[combined_df['Short % of Float'] >= 20]

# Convert 'Market Cap' column to numeric
combined_df['Market Cap'] = pd.to_numeric(combined_df['Market Cap'], errors='coerce')

# Drop columns where Market Cap is less than 300,000,000
combined_df = combined_df[combined_df['Market Cap'] >= 300000000]

# Replace 'Record Date' values
date_mapping = {
    'JanA': '01-11', 'JanB': '01-25',
    'FebA': '02-09', 'FebB': '02-27',
    'MarA': '03-09', 'MarB': '03-24',
    'AprA': '04-12', 'AprB': '04-25',
    'MayA': '05-09', 'MayB': '05-24',
    'JunA': '06-09', 'JunB': '06-27'
}

combined_df['Record Date'] = combined_df['Record Date'].str.replace(r'(\d{4})-(\w+)', lambda m: f'{m.group(1)}-{date_mapping[m.group(2)]}')

# Reset the index
combined_df.reset_index(drop=True, inplace=True)

# Sort by 'Record Date' in ascending order
combined_df.sort_values('Record Date', inplace=True)

# Define the output file path
output_file_path = 'Resources/2023_Short-Interest-Data.csv'

# Export the DataFrame to CSV
combined_df.to_csv(output_file_path, index=False)

# Display the combined dataframe
combined_df

/Users/regulolanz/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:53: FutureWarning: The default value of regex will change from True to False in a future version.


,Company Name,Symbol,Short % of Float,Price,Market Cap,Exchange,Sector,Industry,Record Date
0,Allogene Therapeutics Inc,ALLO,41.77,7.99,1.130905e+09,NAS,Healthcare,Biotechnology,2023-01-11
30,Sunnova Energy International Inc,NOVA,23.58,18.64,2.087121e+09,NY,Technology,Solar,2023-01-11
31,Petmed Express Inc,PETS,20.07,20.71,4.197917e+08,NAS,Healthcare,Pharmaceutical Retailers,2023-01-11
32,Childrens Place Inc (the,PLCE,20.59,40.53,5.945751e+08,NAS,Consumer Cyclical,Apparel Retail,2023-01-11
33,PMV Pharmaceuticals Inc. - Common Stock,PMVP,30.57,8.24,3.710472e+08,NAS,Healthcare,Biotechnology,2023-01-11
...,...,...,...,...,...,...,...,...,...
414,Sonic Automotive Inc,SAH,20.82,48.12,1.422908e+09,NY,Consumer Cyclical,Auto & Truck Dealerships,2023-06-09
413,Root Inc. - common stock,ROOT,32.25,9.98,8.671622e+08,NAS,Financial Services,Insurance - Property & Casualty,2023-06-09
412,Relay Therapeutics Inc. - Common Stock,RLAY,25.44,12.57,1.136705e+09,NAS,Healthcare,Biotechnology,2023-06-09
410,B. Riley Financial Inc,RILY,26.78,44.12,1.199623e+09,NAS,Financial Services,Financial Conglomerates,2023-06-09
